# Dynamic Programming to Optimize Discounted Rewards

Let's consider the weighted finite graph discussed in the lecture on 06/08 and proceed with modeling it. Our objective is to develop a range of algorithms that can efficiently compute the optimal value as well as an optimal policy for this graph.

<center>
    <img width=50% src="graph01.png">
</center>

We will implement three algorithms:
1. value iteration
2. policy iteration
3. linear programming

Let's first start by importing numpy and pulp packages.

In [1]:
!pip install pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 76.5 MB/s eta 0:00:00


In [2]:
import numpy as np
from pulp import *

In our model, we aim to keep it flexible enough to accommodate the modeling of Markov Decision Processes (MDPs). To achieve this, we represent transitions and rewards using three-dimensional matrices of size |S|x|A|x|S|. These matrices provide the transition probabilities and rewards associated with each transition (s, a, s').

In [3]:
# A simple MDP

states = ['s1', 's2']
actions = ['a', 'b']

# transitions[s][a][s'] = probability of the transition (s, a, s')
transitions = np.array([
    [[1.0, 0.0], [0.0, 1.0]],
    [[0.0, 1.0], [1.0, 0.0]]
])

# rewards[s][a][s'] = reward of the transition (s, a, s')
rewards = np.array([
    [[2.0, 0.0], [0.0, 1.0]],
    [[0.0, 5.0], [8.0, 0.0]],
])

discount = 0.8

## Value Iteration
Our first algorithm is value iteration. It is based on Bellman operator defined from the Belmman optimality equations

<center><img width=50% src="bellman.png"></center>




In [4]:
def value_iteration(epsilon=1e-8, max_iterations=1000):
    num_states = len(states)
    num_actions = len(actions)
    V = np.zeros(num_states)

    for _ in range(max_iterations):
        V_prev = np.copy(V)
        for s in range(num_states):
            Q = np.zeros(num_actions)
            for a in range(num_actions):
                for next_state in range(num_states):
                    reward = rewards[s][a][next_state]
                    prob = transitions[s][a][next_state]
                    Q[a] += prob * (reward + discount * V_prev[next_state])
            V[s] = np.max(Q)

        if np.max(np.abs(V - V_prev)) < epsilon:
            break

    return V

# Value Iteration
print("Value Iteration:")
v_values = value_iteration()
print(v_values)

Value Iteration:
[20.99999997 24.99999997]


In [5]:
def compute_value(policy, epsilon=1e-8, max_iterations=1000):
    num_states = len(states)
    num_actions = len(actions)
    V = np.zeros(num_states)

    for _ in range(max_iterations):
        V_prev = np.copy(V)

        for s in range(num_states):
            a = policy[s]
            Q = np.zeros(num_actions)

            for next_state in range(num_states):
                Q[a] += rewards[s][a][next_state] + discount * transitions[s][a][next_state] * V_prev[next_state]      # Q(2): Update this Q[a] properly

            V[s] = Q[a]

        if np.max(np.abs(V - V_prev)) < epsilon:
            break

    return V

In [6]:
def policy_iteration(max_iterations=1000):
    num_states = len(states)
    num_actions = len(actions)
    V = np.zeros(num_states)
    policy = np.zeros(num_states, dtype=int)

    for _ in range(max_iterations):
        V = compute_value(policy)
        policy_stable = True

        for s in range(num_states):
            old_action = policy[s]
            Q = np.zeros(num_actions)

            for a in range(num_actions):
                for next_state in range(num_states):
                    Q[a] = rewards[s][a][next_state] + discount * transitions[s][a][next_state] * V[next_state] # Q(3): Update this value properly

            if (Q[policy[s]] != np.argmax(Q)):
                policy[s] = np.argmax(Q)

            if old_action != policy[s]:
                policy_stable = False

        if policy_stable:
            break

    return V, policy

In [7]:
# Policy Iteration
print("\nPolicy Iteration:")
v_values, policy = policy_iteration()
print("Values:")
print(v_values)
print("Policy:")
print(policy)


Policy Iteration:
Values:
[20.99999996 24.99999996]
Policy:
[1 0]


In [8]:
    def linear_programming():
        num_states = len(states)
        num_actions = len(actions)

        # Create an LP Minimization problem
        lp_prob = LpProblem("LinearProgramming", LpMinimize)

        # Create problem Variables
        V = LpVariable.dicts("V", range(num_states), lowBound=0)

        # Objective Function "Minimize \Sigma_{i=0}^{n-1} V[i]"
        lp_prob += lpSum(V)

        # Constraints:
        # For every transition (s, a, s') we have V[s] >= r(s, a, s') + discount * V[s']
        for s in range(num_states):
            for a in range(num_actions):
                action_sum = 0
                for next_state in range(num_states):
                    prob_val = transitions[s, a, next_state]
                    action_sum += prob_val * (rewards[s, a, next_state] + discount * V[next_state])

                lp_prob += V[s] >= action_sum


        lp_prob.solve(PULP_CBC_CMD(msg=0))

        v_values = np.array([value(V[i]) for i in range(num_states)])

        return v_values

In [9]:
# Linear Programming
print("\nLinear Programming:")
v_values = linear_programming()
print(v_values)


Linear Programming:
[21. 25.]
